In [71]:
import os
import sys
import numpy as np
import scipy.optimize
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit

In [72]:
FIG_FOLDER = 'fig'
SOURCE_FOLDER = os.path.join('data', 'source')
BACKUP_FOLDER = os.path.join('data', 'backup')
print(f"The source folder is: {os.path.abspath(SOURCE_FOLDER)}")
print(f"The figure folder is: {os.path.abspath(FIG_FOLDER)}")
print(f"The backup folder is: {os.path.abspath(BACKUP_FOLDER)}")

# Create folders
for f in SOURCE_FOLDER, FIG_FOLDER, BACKUP_FOLDER:
    os.makedirs(f, exist_ok=True)

The source folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/source
The figure folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/fig
The backup folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/backup


## Decision-making model

In [73]:
class Model:
    
    param_labels = ['distortion', 'precision', 'risk_aversion']
    fit_bounds = [(0.2, 1.8), (0.1, 10.0), (-0.99, 0.99)]

    def __init__(self, param):
        self.distortion, self.precision, self.risk_aversion = param
    
    def p_choice(self, p0, x0, p1, x1, c, *args, **kwargs):

        p = self.p(p0=p0, x0=x0, p1=p1, x1=x1)
        return p[int(c)]

    @classmethod
    def softmax(cls, v, precision):
        return expit(v/precision)

    @staticmethod
    def u(x, risk_aversion):
        if isinstance(x, np.ndarray):
            raise Exception
        else:
            if x >= 0:
                return x ** (1-risk_aversion)
            else:
                return - np.abs(x) ** (1 + risk_aversion)
    
    @classmethod
    def pi(cls, p, alpha):
        if isinstance(p, np.ndarray):
            to_return = np.zeros(p.shape)
            unq_zero = p != 0
            to_return[unq_zero] = np.exp(-(-np.log(p)) ** alpha)
            return to_return
        else:
            if p == 0:
                return 0
            else:
                return np.exp(-(-np.log(p)) ** alpha)

    def p(self, p0, x0, p1, x1):

        v0 = self.pi(p0, self.distortion) * self.u(x0, self.risk_aversion)
        v1 = self.pi(p1, self.distortion) * self.u(x1, self.risk_aversion)
        p = np.zeros(2)
        p[0] = self.softmax(v0-v1, self.precision)
        p[1] = 1 - p[0]
        return p

## Optimization technique

In [74]:
EPS = np.finfo(float).eps

def objective(param, model, data):
    # Since we will look for the minimum, 
    # let's return -LLS instead of LLS

    inst = model(param=param)

    n = len(data)
    ll = np.zeros(n)

    for i, (_, row) in enumerate(data.iterrows()):
        pi = inst.p_choice(**row)
        ll[i] = np.log(pi + EPS)

    return -ll.sum()

def optimize(model, data):

    # Define an init guess
    init_guess = [(b[1] - b[0])/2 for b in model.fit_bounds]

    # Run the optimizer
    res = scipy.optimize.minimize(
        fun=objective,
        x0=init_guess,
        bounds=model.fit_bounds,
        args=(model, data))

    # Make sure that the optimizer ended up with success
    assert res.success

    # Get the best param and best value from the 
    best_param = res.x
    best_value = res.fun

    return best_param, best_value

In [140]:
EPS = np.finfo(float).eps

def fast_objective(param, model, data):
    # Since we will look for the minimum, 
    # let's return -LLS instead of LLS
    
    distortion, precision, risk_aversion = param

    n = len(data)

    sp0 = np.exp(-(-np.log(data.p0.values)) ** distortion)
    sp1 = np.exp(-(-np.log(data.p1.values)) ** distortion)
    
    if np.all(data.x0.values >= 0) and np.all(data.x1.values >= 0):
        su0 = data.x0.values ** (1 - risk_aversion)
        su1 = data.x1.values ** (1 - risk_aversion)
        
    elif np.all(data.x0.values <= 0) and np.all(data.x1.values <= 0):
        su0 = - np.abs(data.x0.values) ** (1 + risk_aversion)
        su1 = - np.abs(data.x1.values) ** (1 + risk_aversion)
    
    else:
        raise ValueError
    
    v0 = sp0 * su0
    v1 = sp1 * su1
    
    delta = v0 - v1
    
    p = np.zeros((2, n))
    p[0] = expit(delta/precision)
    p[1] = 1 - p[0]

    lls = np.log(p[0, data.c.values==0] + EPS).sum()
    lls += np.log(p[1, data.c.values==1] + EPS).sum()
    return -lls

def fast_optimize(model, data):

    # Define an init guess
    init_guess = [(b[1] - b[0])/2 for b in model.fit_bounds]

    # Run the optimizer
    res = scipy.optimize.minimize(
        fun=fast_objective,
        x0=init_guess,
        bounds=model.fit_bounds,
        args=(model, data))

    # Make sure that the optimizer ended up with success
    assert res.success

    # Get the best param and best value from the 
    best_param = res.x
    best_value = res.fun

    return best_param, best_value

In [141]:
def generate_stimuli(n_trial):
    
    possible_p = [0.25, 0.5, 0.75, 1]
    possible_x = np.arange(1, 4)
    p = np.array([np.random.choice(possible_p, size=2, replace=False) for _ in range(n_trial)])
    p = np.sort(p, axis=-1)
    x = np.array([np.random.choice(possible_x, size=2, replace=False) for _ in range(n_trial)])
    x = -np.sort(-x, axis=-1)
    return pd.DataFrame({"p0": p[:, 0], "p1": p[:, 1], "x0": x[:, 0], "x1": x[:, 1]})

In [143]:
def simulate(model, param, n_trial):
    m = Model(param=param)
    stim = generate_stimuli(n_trial=n_trial)
    d = []
    for t, (_, row) in enumerate(stim.iterrows()):
        p = m.p(**row)
        c = int(p[1] > np.random.random())
        d.append({"c": c, **row})
    data = pd.DataFrame(d)
    return data

In [144]:
def fast_simulate(model, param, n_trial):

    data = generate_stimuli(n_trial=n_trial)
    
    distortion, precision, risk_aversion = param

    n = len(data)

    sp0 = np.exp(-(-np.log(data.p0.values)) ** distortion)
    sp1 = np.exp(-(-np.log(data.p1.values)) ** distortion)
    
    su0 = data.x0.values ** (1 - risk_aversion)
    su1 = data.x1.values ** (1 - risk_aversion)
    
    v0 = sp0 * su0
    v1 = sp1 * su1
    
    delta = v0 - v1
    
    p = np.zeros((n, 2))
    p[:, 0] = expit(delta/precision)
    p[:, 1] = 1 - p[:, 0]
    
    c = np.zeros(n, dtype=int)
    r = np.random.random(size=n)
    c[:] = p[:, 1] > r
    
    data["c"] = c
    return data